In [3]:
import numpy as np
import pingouin as pg
import pandas as pd


/usr/local/lib/python3.8/dist-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.8, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/usr/local/lib/python3.8/dist-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Utils

In [2]:
def read_csv(name):
    df = pd.read_csv(f'../data/stats/{name}.csv')
    # pd.set_option('display.max_columns', None)
    # pd.set_option('max_columns', None)
    print(df.head())
    return df

# Review, fig1, test accuracy bad/good

In [15]:
filename = 'corr_tables'
df = read_csv(filename)
pd.options.display.float_format = '{:,.5f}'.format
pg.pairwise_ttests(dv='slope', subject='subject', between='exp_num', within='modality', 
            data=df, padjust='bonf', within_first=False, parametric=True)

# pg.print_table(res, floatfmt='.6f')




   subject  exp_num   slope modality
0        1        1 0.88085       LE
1        2        1 1.09375       LE
2        3        1 0.90403       LE
3        4        1 1.03592       LE
4        5        1 0.91655       LE


,Contrast,modality,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,hedges
0,modality,-,LE,ES,True,True,13.86795,159.00000,two-sided,0.00000,0.00000,bonf,9.258e+25,1.51641
1,modality,-,LE,EE,True,True,7.79477,159.00000,two-sided,0.00000,0.00000,bonf,8.93e+09,0.81647
2,modality,-,ES,EE,True,True,-6.37269,159.00000,two-sided,0.00000,0.00000,bonf,4.742e+06,-0.44898
3,exp_num,-,1,2,False,True,-3.96475,158.00000,two-sided,0.00011,NaN,NaN,191.391,-0.62390
4,modality * exp_num,LE,1,2,False,True,-1.51914,158.00000,two-sided,0.13073,0.39218,bonf,0.493,-0.23905
5,modality * exp_num,ES,1,2,False,True,-4.53675,158.00000,two-sided,0.00001,0.00003,bonf,1520.388,-0.71391
6,modality * exp_num,EE,1,2,False,True,-2.41135,158.00000,two-sided,0.01704,0.05112,bonf,2.427,-0.37945


# Fig 2, test slope bad good

In [61]:
filename = 'review_perf_fig2'
df = read_csv(filename)

pg.pairwise_ttests(dv='value', between=['exp_num', 'split'], subject='id',
            data=df, padjust='bonf', parametric=True)





     value split  exp_num
0  0.05000  poor        2
1  0.15005  poor        2
2  0.01666  poor        1
3 -0.08329  poor        3
4  0.25000  poor        6


,Contrast,exp_num,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,hedges
0,exp_num,-,2,1,False,True,-1.89947,132.72359,two-sided,0.05967,1.00000,bonf,0.919,-0.30798
1,exp_num,-,2,3,False,True,-2.65213,151.24351,two-sided,0.00885,0.18588,bonf,4.238,-0.41916
2,exp_num,-,2,6,False,True,-3.81437,125.06034,two-sided,0.00021,0.00448,bonf,112.285,-0.65360
3,exp_num,-,2,4,False,True,-4.07336,150.28216,two-sided,0.00007,0.00157,bonf,278.435,-0.61868
4,exp_num,-,2,7,False,True,-2.81617,140.00000,two-sided,0.00556,0.11681,bonf,6.391,-0.47012
5,exp_num,-,2,5,False,True,-2.92757,140.00000,two-sided,0.00399,0.08376,bonf,8.473,-0.48872
6,exp_num,-,1,3,False,True,-0.41602,148.18627,two-sided,0.67800,1.00000,bonf,0.185,-0.06619
7,exp_num,-,1,6,False,True,-1.51580,139.86682,two-sided,0.13183,1.00000,bonf,0.515,-0.25061
8,exp_num,-,1,4,False,True,-1.84743,160.49854,two-sided,0.06653,1.00000,bonf,0.812,-0.28688
9,exp_num,-,1,7,False,True,-0.57572,142.00029,two-sided,0.56572,1.00000,bonf,0.207,-0.09385


In [18]:
filename = 'review_accuracy'
df = read_csv(filename)

res = pg.pairwise_ttests(dv='value', between=['modality', 'split'], subject='id',
            data=df, padjust='bonf', parametric=True)

pg.print_table(res, floatfmt='.6f')




   Exp.  Low LE  High LE  % difference LE  Low ES  High ES  % difference ES
0     1    55.0     77.2             22.2    78.9     82.5              3.6
1     1    63.2     79.8             16.6    80.4     83.2              2.9
2     1    71.7     93.3             21.6    81.2     86.3              5.1
3     1    64.9     93.5             28.6    83.5     88.2              4.6
4     1    73.7     92.8             19.0    82.8     85.9              3.1


KeyError: 'value'